In [1]:
# 2025/7/28
# zhangzhong

In [2]:
from datasets import DatasetInfo, load_dataset, load_dataset_builder
from huggingface_hub import login

In [ ]:
# Login using your token
# remove token
login(token="...", add_to_git_credential=True)

In [4]:
wikipedia_builder = load_dataset_builder(
    path="wikimedia/wikipedia",
    name="20231101.en",
)

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

In [5]:
print(wikipedia_builder.info)
wikipedia_info = wikipedia_builder.info
print(wikipedia_info.features)
# how to get all the features key into a list
assert wikipedia_info.features is not None
features_keys = list(wikipedia_info.features.keys())
print(features_keys)
# get the split train
assert wikipedia_info.splits is not None
split_info = wikipedia_info.splits
train_split_info = split_info.get("train")
print(train_split_info)
print(train_split_info.num_bytes)

DatasetInfo(description='', citation='', homepage='', license='', features={'id': Value('string'), 'url': Value('string'), 'title': Value('string'), 'text': Value('string')}, post_processed=None, supervised_keys=None, builder_name='parquet', dataset_name='wikipedia', config_name='20231101.en', version=0.0.0, splits={'train': SplitInfo(name='train', num_bytes=20200062385, num_examples=6407814, shard_lengths=None, dataset_name=None)}, download_checksums=None, download_size=11630929031, post_processing_size=None, dataset_size=20200062385, size_in_bytes=None)
{'id': Value('string'), 'url': Value('string'), 'title': Value('string'), 'text': Value('string')}
['id', 'url', 'title', 'text']
SplitInfo(name='train', num_bytes=20200062385, num_examples=6407814, shard_lengths=None, dataset_name=None)
20200062385


In [6]:
arxiv_builder = load_dataset_builder(
    path="common-pile/arxiv_papers_filtered",
)
print(arxiv_builder.info)
# 对于这种情况要怎么办呢？
# 只能自己提供了

Repo card metadata block was not found. Setting CardData to empty.


DatasetInfo(description='', citation='', homepage='', license='', features=None, post_processed=None, supervised_keys=None, builder_name='json', dataset_name='arxiv_papers_filtered', config_name='default', version=0.0.0, splits=None, download_checksums=None, download_size=None, post_processing_size=None, dataset_size=None, size_in_bytes=None)


In [ ]:
# 写一个函数，只需要传入 path name split
# 就可以得到 features
# 再写一个，传入得到num_bytes
def get_features(path: str, name: str | None, split: str = "train") -> list[str]:
    # builder = load_dataset_builder(path=path, name=name)
    # info: DatasetInfo = builder.info
    # if info.features is None:
    #     raise ValueError("Features are not available.")
    # return list(info.features.keys())
    # 而且不管有没有用streaming 这个方法都可以正确的获得features
    # 卧槽，还得是我呀，哈哈哈！
    ds = load_dataset(path=path, name=name, split=split, streaming=True)
    example = next(iter(ds))
    if not isinstance(example, dict):
        msg = "Example is not a dictionary."
        raise TypeError(msg)
    return list(example.keys())

In [8]:
# 传入咱们需要测试的六个数据集，看看能不能正确的拿到
print(get_features(path="allenai/c4", name="en"))
print(get_features(path="wikimedia/wikipedia", name="20231101.en"))
print(get_features(path="nampdn-ai/tiny-codes", name=None))
print(get_features(path="eminorhan/gutenberg_en", name="chunk_size_2048"))
print(get_features(path="donfu/oa-stackexchange", name=None))
# 我真是服了，为了不提供啊？
print(get_features(path="common-pile/arxiv_papers", name=None))

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

['text', 'timestamp', 'url']


Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

['id', 'url', 'title', 'text']
['prompt', 'main_topic', 'subtopic', 'adjective', 'action_verb', 'scenario', 'target_audience', 'programming_language', 'common_sense_topic', 'idx', 'response']
['text', 'title', 'author', 'author year of birth', 'author year of death', 'language', 'downloads', 'subjects', 'document id', 'type']
['INSTRUCTION', 'RESPONSE', 'SOURCE', 'METADATA']


Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

['id', 'text', 'source', 'created', 'added', 'metadata']


In [9]:
# 然后是获取num_bytes
def get_num_bytes(path: str, name: str | None, split: str = "train") -> int:
    builder = load_dataset_builder(path=path, name=name)
    info: DatasetInfo = builder.info
    if info.splits is None:
        raise ValueError("Splits are not available.")
    split_info = info.splits.get(split)
    if split_info is None:
        raise ValueError(f"Split '{split}' is not available.")
    return split_info.num_bytes

In [10]:
print(get_num_bytes(path="allenai/c4", name="en"))
print(get_num_bytes(path="wikimedia/wikipedia", name="20231101.en"))
print(get_num_bytes(path="nampdn-ai/tiny-codes", name=None))
# print(get_num_bytes(path="eminorhan/gutenberg_en", name="chunk_size_2048"))
print(get_num_bytes(path="donfu/oa-stackexchange", name=None))
# print(get_num_bytes(path="common-pile/arxiv_papers", name=None))
# 果然还是这两个没有。。。
# 这个东西的解决方式就只能是自己手动在代码里写上了，这个真没办法
#

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

828589180707


Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

20200062385
3860932797
6549838664
